# Keras简单入门

Keras的核心数据结构是“模型”，模型是一种组织网络层的方式。Keras中主要的模型是Sequential模型，Sequential是一系列网络层按顺序构成的栈。你也可以查看泛型模型来学习建立更复杂的模型

In [4]:
#Sequential模型如下：
from keras.models import Sequential
model = Sequential()

#将一些网络层通过.add()堆叠起来，就构成了一个模型：
from keras.layers import Dense, Activation

model.add(Dense(output_dim=64, input_dim=100))
model.add(Activation("relu"))
model.add(Dense(output_dim=10))
model.add(Activation("softmax"))

#完成模型的搭建后，我们需要使用.compile()方法来编译模型：
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

编译模型时必须指明损失函数和优化器，如果你需要的话，也可以自己定制损失函数。Keras的一个核心理念就是简明易用同时，保证用户对Keras的绝对控制力度，用户可以根据自己的需要定制自己的模型、网络层，甚至修改源代码。

In [5]:
from keras.optimizers import SGD
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True))

In [ ]:
#完成模型编译后，我们在训练数据上按batch进行一定次数的迭代训练，以拟合网络
#batch_size指的是优化算法 批梯度下降，现在用的比较多的是mini-batch gradient decent(小批的梯度下降)
#model.fit(X_train, Y_train, nb_epoch=5, batch_size=32)

#也可以手动将一个个batch的数据送入网络中训练，这时候需要使用：
model.train_on_batch(X_batch, Y_batch)

#随后，我们可以使用一行代码对我们的模型进行评估，看看模型的指标是否满足我们的要求：
loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)

#或者，我们可以使用我们的模型，对新的数据进行预测：
classes = model.predict_classes(X_test, batch_size=32)
proba = model.predict_proba(X_test, batch_size=32)

# 快速开始Sequential模型

Sequential是多个网络层的线性堆叠
可以通过向Sequential模型传递一个layer的list来构造该模型：

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Activation,LSTM

model = Sequential([
Dense(32, input_dim=784),
Activation('relu'),
Dense(10),
Activation('softmax'),
])

In [9]:
#也可以通过.add()方法一个个的将layer加入模型中：
model = Sequential()
model.add(Dense(32, input_dim=784))
model.add(Activation('relu'))

## 指定输入数据的shape

模型需要知道输入数据的shape，因此，Sequential的第一层需要接受一个关于输入数据shape的参数，后面的各个层则可以自动的推导出中间数据的shape，因此不需要为每个层都指定这个参数。有几种方法来为第一层指定输入数据的shape:
1. 传递一个input_shape的关键字参数给第一层，input_shape是一个tuple类型的数据，其中也可以填入None，如果填入None则表示此位置可能是任何正整数。数据的batch大小不应包含在其中。
2. 传递一个batch_input_shape的关键字参数给第一层，该参数包含数据的batch大小。该参数在指定固定大小batch时比较有用，例如在stateful RNNs中。事实上，Keras在内部会通过添加一个None将input_shape转化为batch_input_shape
3. 有些2D层，如Dense，支持通过指定其输入维度input_dim来隐含的指定输入数据shape。一些3D的时域层支持通过参数input_dim和input_length来指定输入shape。


In [10]:
#下面的三个指定输入数据shape的方法是严格等价的：
model = Sequential()
model.add(Dense(32, input_shape=(784,)))

model = Sequential()
model.add(Dense(32, batch_input_shape=(None, 784)))
# note that batch dimension is "None" here,
# so the model will be able to process batches of any size.</pre>

model = Sequential()
model.add(Dense(32, input_dim=784))

In [13]:
#下面三种方法也是严格等价的：
model = Sequential()
model.add(LSTM(32, input_shape=(10, 64)))

model = Sequential()
model.add(LSTM(32, batch_input_shape=(None, 10, 64)))

model = Sequential()
model.add(LSTM(32, input_length=10, input_dim=64))

## Merge层

多个Sequential可经由一个Merge层合并到一个输出。Merge层的输出是一个可以被添加到新  Sequential的层对象。下面这个例子将两个Sequential合并到一起：

In [9]:
from keras.layers import Merge

left_branch = Sequential()
left_branch.add(Dense(32, input_dim=784))

right_branch = Sequential()
right_branch.add(Dense(32, input_dim=784))

merged = Merge([left_branch, right_branch], mode='concat')

final_model = Sequential()
final_model.add(merged)
final_model.add(Dense(10, activation='softmax'))

![](http://i.imgur.com/LcfRVlr.png)

**Merge层支持一些预定义的合并模式，包括：**
- sum(defualt):逐元素相加
- concat:张量串联，可以通过提供concat_axis的关键字参数指定按照哪个轴进行串联
- mul：逐元素相乘
- ave：张量平均
- dot：张量相乘，可以通过dot_axis关键字参数来指定要消去的轴
- cos：计算2D张量（即矩阵）中各个向量的余弦距离

**这个两个分支的模型可以通过下面的代码训练:**

In [ ]:
final_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
final_model.fit([input_data_1, input_data_2], targets)  # we pass one data array per model input

**也可以为Merge层提供关键字参数mode，以实现任意的变换，例如：**

In [ ]:
merged = Merge([left_branch, right_branch], mode=lambda x: x[0] - x[1])

# 编译

在训练模型之前，我们需要通过compile来对学习过程进行配置。compile接收三个参数：

- 优化器optimizer：该参数可指定为已预定义的优化器名，如rmsprop、adagrad，或一个Optimizer类的对象，详情见optimizers
- 损失函数loss：该参数为模型试图最小化的目标函数，它可为预定义的损失函数名，如categorical_crossentropy、mse，也可以为一个损失函数。详情见objectives
- 指标列表metrics：对分类问题，我们一般将该列表设置为metrics=['accuracy']。指标可以是一个预定义指标的名字,也可以是一个用户定制的函数.指标函数应该返回单个张量,或一个完成metric_name > metric_value映射的字典.请参考性能评估

In [11]:
# for a multi-class classification problem
model.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])

# for a binary classification problem
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['accuracy'])

# for a mean squared error regression problem
model.compile(optimizer='rmsprop',
loss='mse')

# for custom metrices


# for custom metrics
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

def false_rates(y_true, y_pred):
    false_neg = ...
    false_pos = ...
    return {
        'false_neg': false_neg,
        'false_pos': false_pos,
    }

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', mean_pred, false_rates])

# 训练

Keras以Numpy数组作为输入数据和标签的数据类型。训练模型一般使用fit函数

In [12]:
# for a single-input model with 2 classes (binary):
model = Sequential()
model.add(Dense(1, input_dim=784, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# generate dummy data
import numpy as np
data = np.random.random((1000, 784))
labels = np.random.randint(2, size=(1000, 1))

# train the model, iterating on the data in batches
# of 32 samples
model.fit(data, labels, nb_epoch=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 5s - loss: 0.7333 - acc: 0.4900     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 0.7180 - acc: 0.5110     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 0.7146 - acc: 0.5110     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 0.7017 - acc: 0.5310     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 0.6957 - acc: 0.5370     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 0.6824 - acc: 0.5670     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 0.6756 - acc: 0.5890     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 0.6727 - acc: 0.5790     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 0.6672 - acc: 0.5890     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 0.6551 - acc: 0.5970     


In [13]:
# for a multi-input model with 10 classes:

left_branch = Sequential()
left_branch.add(Dense(32, input_dim=784))

right_branch = Sequential()
right_branch.add(Dense(32, input_dim=784))

merged = Merge([left_branch, right_branch], mode='concat')

model = Sequential()
model.add(merged)
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# generate dummy data
import numpy as np
from keras.utils.np_utils import to_categorical
data_1 = np.random.random((1000, 784))
data_2 = np.random.random((1000, 784))

# these are integers between 0 and 9
labels = np.random.randint(10, size=(1000, 1))
# we convert the labels to a binary matrix of size (1000, 10)
# for use with categorical_crossentropy
labels = to_categorical(labels, 10)

# train the model
# note that we are passing a list of Numpy arrays as training data
# since the model has 2 inputs
model.fit([data_1, data_2], labels, nb_epoch=10, batch_size=32)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 2.9234 - acc: 0.1050     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 2.6060 - acc: 0.1270     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 2.4940 - acc: 0.1310     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 2.3536 - acc: 0.1670     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 2.3049 - acc: 0.1840     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 2.1454 - acc: 0.2420     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 2.0661 - acc: 0.3030     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 1.9465 - acc: 0.3210     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 1.8940 - acc: 0.3370     
Epoch 10/10
1000/1000 [==============================] - 0s - loss: 1.8513 - acc: 0.3750     


# 例子

在Keras代码包的examples文件夹中，你将找到使用真实数据的示例模型：

- CIFAR10 小图片分类：使用CNN和实时数据提升
- IMDB 电影评论观点分类：使用LSTM处理成序列的词语
- Reuters（路透社）新闻主题分类：使用多层感知器（MLP）
- MNIST手写数字识别：使用多层感知器和CNN
- 字符级文本生成：使用LSTM ...

**基于多层感知器的softmax多分类：**

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, input_dim=20, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(64, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(10, init='uniform'))
model.add(Activation('softmax'))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(X_train, y_train,
          nb_epoch=20,
          batch_size=16)
score = model.evaluate(X_test, y_test, batch_size=16)   


Using TensorFlow backend.


NameError: name 'X_train' is not defined

**相似MLP的另一种实现：**

In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

**用于二分类的多层感知器：**

In [2]:
model = Sequential()
model.add(Dense(64, input_dim=20, init='uniform', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])